Загрузка библиотек

In [1]:
import cv2
from ultralytics import YOLO
import numpy as np
import matplotlib.pyplot as plt
#import ipywidgets as widgets           данные библиотеки нужны для интерактивного вывода, где нужно "поиграться" с размерами, метриками 
#from ipywidgets import interact 
import torch

Загрузка модели

In [2]:
model = YOLO('C:/Users/la-li/Desktop/STARS/OBB/yolo8s/weights/best.pt')
             

Инференс модели. Просмотр изображений

In [13]:


# Загрузка изображения
img = cv2.imread('C:/Users/la-li/Desktop/STARS/OBB/images_test/0050_test.jpg')

# Инференс с использованием модели YOLO OBB
results = model(img, imgsz=1000, iou=0.4, conf=0.2, verbose=False)

# Переменные для масштабирования и перемещения
scale = 1.0
x_offset, y_offset = 0, 0
start_x, start_y = 0, 0
dragging = False
window_name = 'Image'
window_height = img.shape[0]
window_width = img.shape[1]

# повернутые боксы

def draw_rotated_bbox(image, bbox, scale, x_offset, y_offset):
    bbox = bbox.cpu().numpy()
    cx, cy, w, h, angle = bbox
    
    # Применение масштабирования и смещения
    cx = cx * scale - x_offset
    cy = cy * scale - y_offset
    w = w * scale
    h = h * scale
    
    rect = ((int(cx), int(cy)), (int(w), int(h)), (angle * 180 / np.pi))
    
    # Преобразование центра и размера bbox в целочисленные значения
    box = cv2.boxPoints(rect)
    box = box.astype(int)
    cv2.drawContours(image, [box], 0, (0, 255, 0), 1)  # Зеленая рамка

    rect = ((int(cx), int(cy)), (int(w), int(h)), int(0))
    
    # Преобразование центра и размера bbox в целочисленные значения
    box = cv2.boxPoints(rect)
    box = box.astype(int)
    cv2.drawContours(image, [box], 0, (255, 255, 255), 1)

# Функция для отображения изображения с учетом масштаба и смещения
def display_image():
    global img, scale, x_offset, y_offset, window_width, window_height
    h, w = img.shape[:2]
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(img, (new_w, new_h))
    display_img = np.zeros((window_height, window_width, 3), dtype=np.uint8)

    x_start = max(0, -x_offset)
    y_start = max(0, -y_offset)
    x_end = min(window_width, new_w - x_offset)
    y_end = min(window_height, new_h - y_offset)

    img_x_start = max(0, x_offset)
    img_y_start = max(0, y_offset)
    img_x_end = img_x_start + (x_end - x_start)
    img_y_end = img_y_start + (y_end - y_start)

    display_img[y_start:y_end, x_start:x_end] = resized[img_y_start:img_y_end, img_x_start:img_x_end]

    # Отрисовка bbox на изображении
    for bbox in results[0].obb.xywhr:
        draw_rotated_bbox(display_img, bbox, scale, x_offset, y_offset)
        
        # Добавление подписи угла поворота
        angle = bbox[4] * 180 / np.pi
        label = f"{angle:.4f} grad"
        cv2.putText(display_img, label, (int(bbox[0] * scale - x_offset), int(bbox[1] * scale + bbox[3] * scale / 2 - y_offset + 20)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
    
    cv2.imshow(window_name, display_img)

# Обработка событий мыши
def mouse_callback(event, x, y, flags, param):
    global scale, x_offset, y_offset, start_x, start_y, dragging

    if event == cv2.EVENT_MOUSEWHEEL:
        old_scale = scale
        if flags > 0:
            scale *= 1.1  # Увеличение
        else:
            scale /= 1.1  # Уменьшение

        # Обновление смещения для центра масштабирования
        x_offset = int((x_offset + x - window_width // 2) * scale / old_scale + window_width // 2 - x)
        y_offset = int((y_offset + y - window_height // 2) * scale / old_scale + window_height // 2 - y)
        display_image()

    elif event == cv2.EVENT_LBUTTONDOWN:
        dragging = True
        start_x, start_y = x, y

    elif event == cv2.EVENT_MOUSEMOVE:
        if dragging:
            x_offset += (start_x - x)
            y_offset += (start_y - y)
            start_x, start_y = x, y
            display_image()

    elif event == cv2.EVENT_LBUTTONUP:
        dragging = False

# Настройка окна и обратного вызова
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, window_width, window_height)
cv2.setMouseCallback(window_name, mouse_callback)
display_image()

# Ожидание закрытия окна
cv2.waitKey(0)
cv2.destroyAllWindows()


WARNING  imgsz=[1000] must be multiple of max stride 32, updating to [1024]


Расчеты 

In [11]:
# Вытаскиваем координаты 
x, y, w, h, r = results[0].obb.xywhr[0]
# Рассчитываем начальные и конечные координаты 
x_0 = x - 0.5*(w*np.cos(r) - h * np.cos(r))
x_1 = x + 0.5*(w*np.cos(r) - h * np.cos(r))
y_0 = y + 0.5*(w*np.sin(r) - h * np.sin(r))
y_1 = y - 0.5*(w*np.cos(r) - h * np.cos(r))
# Записываем длину(по Евкликодову расстоянию), ширину, угол направления
length = np.sqrt((x_1 - x_0)**2 + (y_1 - y_0)**2)
width = w 
print((x_0, y_0), (x_1, y_1))
print(length)
print(w)

(tensor(398.1362), tensor(158.1746)) (tensor(407.6489), tensor(145.1530))
tensor(16.1262)
tensor(23.2924)


Алгоритм нахождения сред яркости объекта

In [11]:
# Функция для сегментации объекта и вычисления средней яркости
def calculate_average_brightness(image, bbox):
    cx, cy, w, h, angle = bbox
    rect = ((int(cx), int(cy)), (int(w), int(h)), (angle * 180 / np.pi))
    box = cv2.boxPoints(rect).astype(int)

    mask = np.zeros_like(image)
    cv2.fillPoly(mask, [box], 255)

    # Наложение маски на оригинальное изображение
    masked_image = cv2.bitwise_and(image, mask)

    # Применение Canny для выделения границ
    edges = cv2.Canny(masked_image, 50, 150)

    # Нахождение контуров
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Создание маски на основе контуров
    object_mask = np.zeros_like(image)
    cv2.drawContours(object_mask, contours, -1, 255, thickness=cv2.FILLED)

    object_pixels = image[object_mask > 0]
    if len(object_pixels) > 0:
        average_brightness = np.mean(object_pixels)
    else:
        average_brightness = 0
    return average_brightness

# Основная функция
def main(image_path, model):
    # Загрузка изображения
    img = cv2.imread(image_path)

    # Инференс с использованием модели YOLO OBB
    results = model(img, imgsz=1000, iou=0.4, conf=0.2, verbose=False)

    # Перебор всех найденных объектов и вычисление средней яркости
    for i, bbox in enumerate(results[0].obb.xywhr):
        bbox = bbox.cpu().numpy()
        avg_brightness = calculate_average_brightness(img, bbox)
        print(f"Средняя яркость объекта {i+1}: {avg_brightness:.2f}")

main('C:/Users/la-li/Desktop/STARS/OBB/images_test/0050_test.jpg', model)


WARNING  imgsz=[1000] must be multiple of max stride 32, updating to [1024]


Средняя яркость объекта 1: 67.86
Средняя яркость объекта 2: 124.54
Средняя яркость объекта 3: 73.07
Средняя яркость объекта 4: 72.32
Средняя яркость объекта 5: 69.18
Средняя яркость объекта 6: 61.72
Средняя яркость объекта 7: 66.94
Средняя яркость объекта 8: 65.30


Проверка угла звезды


In [16]:
for res in results[0].obb.xywhr:
    print(res[4]*180/np.pi)

tensor(60.0814)
tensor(61.2092)
tensor(60.5280)
tensor(60.6757)
tensor(58.1387)
tensor(61.7914)
tensor(27.0251)
tensor(151.6768)


Экспорт параметров объекта

In [20]:
results[0].obb.xywhr

tensor([[4.0289e+02, 1.4991e+02, 2.3292e+01, 4.2201e+00, 1.0486e+00],
        [3.5119e+02, 4.4389e+02, 2.5080e+01, 4.3949e+00, 1.0683e+00],
        [6.2288e+02, 9.2456e+01, 1.9954e+01, 3.8999e+00, 1.0564e+00],
        [8.4680e+02, 1.4924e+02, 2.0642e+01, 3.7089e+00, 1.0590e+00],
        [4.7513e+01, 2.9733e+02, 2.1030e+01, 5.2711e+00, 1.0147e+00],
        [7.0525e+02, 6.3597e+02, 2.1401e+01, 4.4021e+00, 1.0785e+00],
        [7.5568e+02, 5.6065e+02, 1.8786e+01, 4.3503e+00, 4.7168e-01],
        [3.5879e+02, 3.8044e+02, 1.7103e+01, 3.6754e+00, 2.6473e+00]])